<a href="https://colab.research.google.com/github/skaumbdoallsaws-coder/AI-Drawing-Inspector/blob/main/ai_inspector_v2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# AI Engineering Drawing Inspector v2.0

**Single-File QC Pipeline**

Outputs:
1. `ResolvedPartIdentity.json`
2. `DrawingEvidence.json`
3. `DiffResult.json`
4. `QCReport.md`

In [ ]:
# Cell 1: Install Dependencies
!pip install -q pymupdf opencv-python-headless jsonschema pillow pytesseract
!pip install -q accelerate qwen-vl-utils bitsandbytes
!pip install -q git+https://github.com/huggingface/transformers
!apt-get install -y poppler-utils tesseract-ocr > /dev/null 2>&1
print("Dependencies installed!")

In [ ]:
# Cell 2: Imports and Configuration
import os, json, re, gc
import torch
import fitz
import numpy as np
from PIL import Image
from pathlib import Path
from dataclasses import dataclass, field, asdict
from typing import List, Dict, Any, Optional, Tuple
from datetime import datetime

# Configuration
DRAWING_PDF_PATH = ""
SOLIDWORKS_JSON_DIR = "sw_json_library"
OUTPUT_DIR = "qc_output"
os.makedirs(OUTPUT_DIR, exist_ok=True)

print(f"PyTorch: {torch.__version__}")
print(f"CUDA: {torch.cuda.is_available()}")

In [ ]:
# Cell 3: BOM-Robust JSON Loader
def load_json_robust(filepath) -> Tuple[Optional[Dict], Optional[str]]:
    """Load JSON with BOM handling. Tries: utf-8-sig, utf-8, latin-1"""
    filepath = Path(filepath)
    for enc in ['utf-8-sig', 'utf-8', 'latin-1']:
        try:
            with open(filepath, 'r', encoding=enc) as f:
                return json.load(f), None
        except UnicodeDecodeError:
            continue
        except json.JSONDecodeError as e:
            if 'BOM' in str(e) and enc == 'utf-8':
                continue
            return None, f"JSON error: {str(e)[:50]}"
        except Exception as e:
            return None, f"Error: {str(e)[:50]}"
    return None, "Failed all encodings"

print("load_json_robust defined")

In [ ]:
# Cell 4: PDF Rendering
@dataclass
class PageArtifact:
    pageIndex0: int
    page: int
    image: Image.Image
    width: int
    height: int
    dpi: int
    direct_text: Optional[str] = None

def render_pdf(pdf_path: str, dpi: int = 300) -> List[PageArtifact]:
    """Render first page of PDF to image."""
    artifacts = []
    doc = fitz.open(pdf_path)
    page = doc.load_page(0)
    zoom = dpi / 72.0
    pix = page.get_pixmap(matrix=fitz.Matrix(zoom, zoom), alpha=False)
    img = Image.frombytes("RGB", [pix.width, pix.height], pix.samples)
    direct_text = page.get_text("text")

    artifacts.append(PageArtifact(
        pageIndex0=0, page=1, image=img,
        width=pix.width, height=pix.height, dpi=dpi,
        direct_text=direct_text if len(direct_text.strip()) > 10 else None
    ))
    doc.close()
    print(f"Rendered: {pix.width}x{pix.height}px")
    return artifacts

print("render_pdf defined")

In [5]:
# Cell 5: SolidWorks JSON Library
@dataclass
class SwPartEntry:
    json_path: str
    part_number: str
    filename_stem: str = ""
    data: Dict[str, Any] = field(default_factory=dict)

class SwJsonLibrary:
    def __init__(self):
        self.by_part_number: Dict[str, SwPartEntry] = {}
        self.by_filename: Dict[str, SwPartEntry] = {}
        self.all_entries: List[SwPartEntry] = []

    def _normalize(self, s: str) -> str:
        return re.sub(r'[-\s_]', '', str(s or '')).lower()

    def load_from_directory(self, directory: str):
        json_files = list(Path(directory).glob("**/*.json"))
        print(f"Found {len(json_files)} JSON files")

        for jp in json_files:
            data, err = load_json_robust(jp)
            if data is None:
                continue
            pn = data.get('identity', {}).get('partNumber', '')
            entry = SwPartEntry(str(jp), pn, jp.stem, data)
            self.all_entries.append(entry)
            if pn:
                self.by_part_number[pn] = entry
                self.by_part_number[self._normalize(pn)] = entry
            self.by_filename[jp.stem] = entry
            self.by_filename[self._normalize(jp.stem)] = entry
        print(f"Loaded {len(self.all_entries)} files")

    def lookup(self, candidate: str) -> Optional[SwPartEntry]:
        if not candidate:
            return None
        norm = self._normalize(candidate)
        return self.by_part_number.get(candidate) or self.by_part_number.get(norm) or \
               self.by_filename.get(candidate) or self.by_filename.get(norm)

sw_library = SwJsonLibrary()
print("SwJsonLibrary defined")

SwJsonLibrary defined


In [ ]:
# Cell 6: Part Identity Resolution (Robust Matching)

@dataclass
class ResolvedPartIdentity:
    partNumber: str
    confidence: float
    source: str
    swJsonPath: Optional[str] = None
    candidates_tried: List[str] = field(default_factory=list)

def clean_filename(filename: str) -> str:
    """Remove known suffixes like Paint, REV, etc."""
    cleaned = re.sub(r'[\s_]*(Paint|PAINT)$', '', filename, flags=re.IGNORECASE)
    return cleaned.strip()

def extract_pn_candidates(filename: str) -> List[str]:
    """
    Extract potential part number candidates from filename.
    Handles: 1013572_01, 101357201-03, 314884W_0, 046-935-REV-A
    Returns list of candidates (most specific to least).
    """
    name_no_ext = os.path.splitext(filename)[0]
    # Remove duplicate markers like (1), (2)
    name_no_ext = re.sub(r'\s*\(\d+\)$', '', name_no_ext)
    cleaned = clean_filename(name_no_ext)
    parts = re.split(r'[\s_]+', cleaned)
    
    if not parts:
        return []
    
    base = parts[0]
    candidates = []
    
    # 1. Base as-is
    candidates.append(base)
    
    # 2. Without hyphens
    base_no_hyphen = base.replace('-', '')
    if base_no_hyphen != base:
        candidates.append(base_no_hyphen)
    
    # 3. Remove letter suffixes (046-935A -> 046-935)
    if base and base[-1].isalpha() and len(base) > 1:
        candidates.append(base[:-1])
        candidates.append(base[:-1].replace('-', ''))
    
    # 4. Handle revision pattern (046-935-01 -> 046-935)
    rev_match = re.match(r'^(.+)-(\d{1,2})$', base)
    if rev_match:
        main_part = rev_match.group(1)
        candidates.append(main_part)
        candidates.append(main_part.replace('-', ''))
    
    # 5. Handle REV suffix (046-935-REV-A -> 046-935)
    rev_alpha = re.match(r'^(.+?)[-_]?REV[-_]?[A-Z0-9]*$', base, re.IGNORECASE)
    if rev_alpha:
        candidates.append(rev_alpha.group(1))
        candidates.append(rev_alpha.group(1).replace('-', ''))
    
    # 6. Peeling - progressively remove trailing digits
    temp = base_no_hyphen
    while len(temp) > 5:
        temp = temp[:-1]
        candidates.append(temp)
    
    # Remove duplicates, preserve order
    seen = set()
    unique = []
    for c in candidates:
        if c and c not in seen:
            seen.add(c)
            unique.append(c)
    
    return unique

def resolve_part_identity(pdf_path: str, artifacts: List[PageArtifact], sw_lib: SwJsonLibrary) -> ResolvedPartIdentity:
    """Resolve part identity using robust filename matching."""
    filename = os.path.basename(pdf_path)
    candidates = extract_pn_candidates(filename)
    
    # Try each candidate against SW library
    for candidate in candidates:
        entry = sw_lib.lookup(candidate)
        if entry:
            return ResolvedPartIdentity(
                partNumber=entry.part_number or candidate,
                confidence=1.0,
                source="filename+sw",
                swJsonPath=entry.json_path,
                candidates_tried=candidates
            )
    
    # Try PDF embedded text
    for art in artifacts:
        if art.direct_text:
            text_candidates = extract_pn_candidates(art.direct_text[:200])
            for candidate in text_candidates[:5]:
                entry = sw_lib.lookup(candidate)
                if entry:
                    return ResolvedPartIdentity(
                        partNumber=entry.part_number or candidate,
                        confidence=0.8,
                        source="pdf_text+sw",
                        swJsonPath=entry.json_path,
                        candidates_tried=candidates + text_candidates[:5]
                    )
    
    # Fallback - use first candidate or filename stem
    fallback_pn = candidates[0] if candidates else Path(pdf_path).stem
    return ResolvedPartIdentity(
        partNumber=fallback_pn,
        confidence=0.3,
        source="fallback",
        swJsonPath=None,
        candidates_tried=candidates
    )

print("resolve_part_identity defined (robust matching)")

In [ ]:
# Cell 7: Load SolidWorks Library (Upload ZIP)
from google.colab import files
import zipfile

if not os.path.exists(SOLIDWORKS_JSON_DIR) or not list(Path(SOLIDWORKS_JSON_DIR).glob("*.json")):
    print("Upload your sw_json_library.zip file:")
    uploaded = files.upload()
    
    for filename in uploaded:
        if filename.endswith('.zip'):
            print(f"Extracting {filename}...")
            with zipfile.ZipFile(filename, 'r') as z:
                z.extractall(SOLIDWORKS_JSON_DIR)
            print(f"Extracted to {SOLIDWORKS_JSON_DIR}")
            break

sw_library.load_from_directory(SOLIDWORKS_JSON_DIR)
print(f"Library ready: {len(sw_library.all_entries)} parts indexed")

In [ ]:
# Cell 8: Upload and Render PDF Drawing
from google.colab import files
from IPython.display import display

print("Upload your PDF drawing:")
uploaded = files.upload()

for filename in uploaded:
    if filename.lower().endswith('.pdf'):
        DRAWING_PDF_PATH = filename
        break

print(f"Processing: {DRAWING_PDF_PATH}")
artifacts = render_pdf(DRAWING_PDF_PATH)

# Display the rendered image
if artifacts:
    display(artifacts[0].image.resize((800, int(800 * artifacts[0].height / artifacts[0].width))))

In [ ]:
# Cell 9: Resolve Part Identity
part_identity = resolve_part_identity(DRAWING_PDF_PATH, artifacts, sw_library)

print("="*50)
print("RESOLVED PART IDENTITY")
print("="*50)
print(f"Part Number:  {part_identity.partNumber}")
print(f"Confidence:   {part_identity.confidence}")
print(f"Source:       {part_identity.source}")
print(f"SW JSON:      {part_identity.swJsonPath or 'Not found'}")
print(f"Candidates:   {part_identity.candidates_tried[:5]}")

# Save to output
identity_out = os.path.join(OUTPUT_DIR, "ResolvedPartIdentity.json")
with open(identity_out, 'w') as f:
    json.dump(asdict(part_identity), f, indent=2)
print(f"\nSaved: {identity_out}")

In [ ]:
# Cell 10: Load LightOnOCR-2 and Run OCR
from transformers import AutoProcessor, AutoModelForVision2Seq
from google.colab import userdata

# Clear GPU memory
gc.collect()
if torch.cuda.is_available():
    torch.cuda.empty_cache()

# Get HF token
try:
    hf_token = userdata.get('HF_TOKEN')
except:
    hf_token = None

print("Loading LightOnOCR-2-1B...")
ocr_device = "cuda" if torch.cuda.is_available() else "cpu"
ocr_dtype = torch.bfloat16 if ocr_device == "cuda" else torch.float32

ocr_processor = AutoProcessor.from_pretrained(
    "lightonai/LightOnOCR-2-1B",
    token=hf_token,
    trust_remote_code=True
)

ocr_model = AutoModelForVision2Seq.from_pretrained(
    "lightonai/LightOnOCR-2-1B",
    torch_dtype=ocr_dtype,
    token=hf_token,
    trust_remote_code=True
).to(ocr_device)

print(f"LightOnOCR-2 loaded: {ocr_model.get_memory_footprint() / 1e9:.2f} GB")

def run_lighton_ocr(image: Image.Image) -> List[str]:
    """Run LightOnOCR-2 on image, return list of text lines."""
    global ocr_model, ocr_processor, ocr_device, ocr_dtype
    
    img = image.convert("RGB")
    conversation = [{"role": "user", "content": [{"type": "image", "image": img}]}]
    
    inputs = ocr_processor.apply_chat_template(
        conversation, add_generation_prompt=True, tokenize=True,
        return_dict=True, return_tensors="pt"
    )
    inputs = {k: v.to(device=ocr_device, dtype=ocr_dtype) if v.is_floating_point() else v.to(ocr_device) for k, v in inputs.items()}
    
    with torch.no_grad():
        output_ids = ocr_model.generate(**inputs, max_new_tokens=2048)
    
    generated_ids = output_ids[0, inputs["input_ids"].shape[1]:]
    output_text = ocr_processor.decode(generated_ids, skip_special_tokens=True)
    
    return [line.strip() for line in output_text.split("\n") if line.strip()]

# Run OCR on the drawing
print("Running OCR on drawing...")
ocr_lines = run_lighton_ocr(artifacts[0].image)
print(f"OCR extracted {len(ocr_lines)} lines")
print("\nFirst 10 lines:")
for line in ocr_lines[:10]:
    print(f"  {line}")